### Catching ThrottlingException exceptions when using a low-level boto3 client

#### Quotas for Amazon Bedrock


Your AWS account has default quotas, formerly referred to as limits, for each AWS service. Unless otherwise noted, each quota is Region-specific within your AWS account. Some quotas may be adjustable. The following list explains the meaning of the Adjustable through Service Quotas column in the following tables:

https://docs.aws.amazon.com/bedrock/latest/userguide/quotas.html

In [63]:
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'

In [64]:
max_tokens_to_sample = 200

system_message = f"You are a long and high-quality story teller. Make the story longer than {max_tokens_to_sample}"

messages = [
  {"role": "user", "content": """
  Rex and Charlie were best friends who did everything together. 
  They lived next door to each other with their human families and spent all day playing in the backyard. 
  Rex was a golden retriever, always happy and eager for fun. Charlie was a German shepherd, more serious but very loyal.  
  Every morning, Rex and Charlie would wake up and bark excitedly, ready to start the day's adventures. 
  Their families would let them out into the backyard and they'd run around chasing each other and sniffing for interesting smells. 
  After tiring themselves out, they'd nap in the shade of the big oak tree, Rex's tail still thumping contentedly even in his sleep. 
  """
  }
]

In [65]:
print(system_message)
print(messages)

You are a long and high-quality story teller. Make the story longer than 200
[{'role': 'user', 'content': "\n  Rex and Charlie were best friends who did everything together. \n  They lived next door to each other with their human families and spent all day playing in the backyard. \n  Rex was a golden retriever, always happy and eager for fun. Charlie was a German shepherd, more serious but very loyal.  \n  Every morning, Rex and Charlie would wake up and bark excitedly, ready to start the day's adventures. \n  Their families would let them out into the backyard and they'd run around chasing each other and sniffing for interesting smells. \n  After tiring themselves out, they'd nap in the shade of the big oak tree, Rex's tail still thumping contentedly even in his sleep. \n  "}]


In [56]:
import boto3
import json
import time 

# Create a Boto3 client for Bedrock Runtime
bedrock_client = boto3.client(service_name='bedrock-runtime')

# Define the prompt and other parameters
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
accept = 'application/json'
contentType = 'application/json'

In [70]:
from botocore.exceptions import ClientError

# Measure function execution time
start_time = time.time()  # Get the current time before calling the function

try:
    # Invoke the model
    response = bedrock_client.invoke_model(
        modelId=modelId,
        accept=accept,
        contentType=contentType,
        body=json.dumps({
            "anthropic_version": "bedrock-2023-05-31", 
            "messages": messages, 
            "system": system_message,    
            "max_tokens": 300, 
            "temperature": 0.7, 
            "top_p": 0.9})
    )
except ClientError as e:
    # Check if the error is a ThrottlingException
    if e.response['Error']['Code'] == 'ThrottlingException':
        print(f"Throttling Exception occurred: {e}")
        
        # You can add additional error handling logic here, such as retrying with exponential backoff or logging
        print("You can add additional error handling logic here, such as retrying with exponential backoff or logging")
    else:
        # Handle other client errors
        print(f"An error occurred: {e}")
except Exception as e:
    # Handle other exceptions here
    print(f"An error occurred: {e}")
else:
    # Code executed if no exception occurred
    end_time = time.time()  # Get the current time after the function call
    execution_time = end_time - start_time  # Calculate the execution time
    # Process the response
    response_body = json.loads(response.get('body').read())
    print(response_body['content'][0]['text'])    
    print(f"Function execution time: {execution_time} seconds")

Throttling Exception occurred: An error occurred (ThrottlingException) when calling the InvokeModel operation (reached max retries: 4): Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again.
You can add additional error handling logic here, such as retrying with exponential backoff or logging
